In [5]:
from feature_engineer import feature_engineer
import pickle
import numpy as np

/home/weijiesun/anaconda3/envs/featxgcat2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
obs_id = 'rcpt_MR_obs_id' 

interim_datadir = '../data/interim/'    
proc_datadir = '../data/processed/'

with open(interim_datadir + 'inpamb_featprocess_0216_MR_obs_200420.pickle', 'rb') as handle:
    inp_amb_dict = pickle.load(handle)
    
for df in inp_amb_dict: 
    inp_amb_dict[df].reset_index(inplace = True, drop = True)

#add record ids
inp_amb_dict['inp_dx']['dx_event_id'] = [(inp_amb_dict['inp_dx']['event_id'][i] + "_" + str(i)) for i in inp_amb_dict['inp_dx'].index]
inp_amb_dict['inp_prvd']['prvd_event_id'] = [(inp_amb_dict['inp_prvd']['event_id'][i] + "_" + str(i)) for i in inp_amb_dict['inp_prvd'].index]
inp_amb_dict['inp_intrv']['intrv_event_id'] = [(inp_amb_dict['inp_intrv']['event_id'][i] + "_" + str(i)) for i in inp_amb_dict['inp_intrv'].index]

inp_amb_dict['amb_dx']['dx_event_id'] = [(inp_amb_dict['amb_dx']['event_id'][i] + "_" + str(i)) for i in inp_amb_dict['amb_dx'].index]
inp_amb_dict['amb_intrv']['intrv_event_id'] = [(inp_amb_dict['amb_intrv']['event_id'][i] + "_" + str(i)) for i in inp_amb_dict['amb_intrv'].index]

#identify removable vars
ll = list(inp_amb_dict['amb_visits'].columns)
print(np.sort(np.array(ll)[np.array(['GRPR' in i for i in ll])]))

remvars = {}
#remvars['amb'] = ['SEPI_CACS_AGE_CAT_CODE','SEPI_CAS_AGE_CAT_CODE','SEPI_MODE_SRVC_CODE','SEPI_REC_GRPR_ID','SEPI_GRPR_ID','SEPI_MODE_SRVC_CODE'] #REC_SUBM_FAC_ID, Total_Cost
remvars['amb'] = ['SEPI_MODE_SRVC_CODE','SEPI_GRPR_ID','SEPI_MODE_SRVC_CODE']
remvars['inp'] = ['CMG_Age_Category_2013_016','SEPI_GRPR_DESC','SEPI_REC_GRPR_ID']
#optional add: #SEPI_CMG_CODE, REC_SUBM_FAC_ID, Total_Cost
inp_amb_dict['inp_visits'].drop(columns=remvars['inp'], axis=1, inplace=True)
inp_amb_dict['amb_visits'].drop(columns=remvars['amb'], axis=1, inplace=True)

/home/weijiesun/anaconda3/envs/featxgcat2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['SEPI_GRPR_ID']


In [7]:
#remove columns with single levels
remvars['inp'] = ['HLTH_DX_TYPE_CODE_MR','HLTH_DX_CODE_VER_NUM_CODE' ,'PRVD_ROLE_CODE_1']
inp_amb_dict['inp_visits'].drop(columns=remvars['inp'], axis=1, inplace=True)

#mode impute the blood columns with small number of nans and convert to bool type
bl_cols = ['HLTH_STATUS_BLD_OTH_CMPTS_FLAG', 'HLTH_STATUS_BLD_RED_CELLS_FLAG','HLTH_STATUS_BLOOD_ALBUMIN_FLAG',
'HLTH_STATUS_BLOOD_PLASMA_FLAG','HLTH_STATUS_BLOOD_PLAT_FLAG','HLTH_STATUS_BLOOD_TRNSFSN_FLAG']
inp_amb_dict['inp_visits'][bl_cols].fillna((inp_amb_dict['inp_visits'][bl_cols].mode()), inplace=True)
inp_amb_dict['inp_visits'][bl_cols] = (inp_amb_dict['inp_visits'][bl_cols] == b'Y').astype(bool)

inp_amb_dict['inp_visits']['SE_CODE_VER_NUM_CODE'] = (inp_amb_dict['inp_visits']['SE_CODE_VER_NUM_CODE'] == b'CCI').astype(bool)


/home/weijiesun/anaconda3/envs/featxgcat2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
#feature extraction using feattools
import featuretools as ft
import featuretools.variable_types as vtypes
import pandas as pd
#create dict for index and time index
es_index_dict = dict(zip(inp_amb_dict.keys(),['event_id','dx_event_id','intrv_event_id','prvd_event_id','event_id','dx_event_id','intrv_event_id']))
es_tindex_dict = dict(zip(inp_amb_dict.keys(),['SEPI_START_DATE',None,'SE_INTRV_DATE_PR_OTH',None,'SEPI_START_DATE',None,None]))
es_index_dict,es_tindex_dict

es = ft.EntitySet('inp_amb')

for reg in inp_amb_dict:
    es = es.entity_from_dataframe(entity_id=reg,
                                  dataframe=inp_amb_dict[reg],
                                  index=es_index_dict[reg],
                                  time_index=es_tindex_dict[reg])
    
#for combined inp and amb obs
inp_amb_obs = pd.DataFrame({obs_id:np.union1d(inp_amb_dict['inp_visits'][obs_id].unique(), inp_amb_dict['amb_visits'][obs_id].unique())})
es = es.entity_from_dataframe(entity_id='inp_amb_obs',
                                  dataframe=inp_amb_obs,
                                  index=obs_id)
for reg in ['inp_visits','amb_visits']:
    es = es.add_relationship(ft.Relationship(es['inp_amb_obs'][obs_id],es[reg][obs_id]))
    
rel1 = ['inp_visits','inp_visits','inp_visits','amb_visits','amb_visits']
rel2 = ['inp_dx','inp_intrv','inp_prvd','amb_dx','amb_intrv']

for i in range(len(rel1)):
    es = es.add_relationship(ft.Relationship(es[rel1[i]]["event_id"],es[rel2[i]]["event_id"]))

/home/weijiesun/anaconda3/envs/featxgcat2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:


def sel_coperct(var, co_perct, fig = False):
    #return unique elements that have minimum precent frequency
    co_perct = co_perct/100
    bl = var.value_counts()>(len(var)*(co_perct))
    print ()
    print (var.head())
    print ('Number of elements : '+str(len(var)))
    print ('Number of unique elements : '+str(len(bl)))
    print ('Minimum thrshold frequency: '+str(len(var)*co_perct))
    var_sel = bl[bl==True].index.tolist()
    print ('Number of unique elements selected : '+str(len(var_sel)))
    if fig == True:
        var.value_counts().plot(kind='bar', logy=True)
        plt.show()
    return var_sel

varsel_inp_dx = sel_coperct(inp_amb_dict['inp_dx']['HLTH_DX_CODE_MR_OTH'], 0.1) 
varsel_inp_dx_type = sel_coperct(inp_amb_dict['inp_dx']['HLTH_DX_TYPE_CODE_MR_OTH'], 0.1) 
varsel_inp_prvd = sel_coperct(inp_amb_dict['inp_prvd']['Prvd_Spec_Grp_MR_Int'], 0.1)
varsel_inp_intrv = sel_coperct(inp_amb_dict['inp_intrv']['SE_INTRV_CODE_PR_OTH'], 0.1)
varsel_amb_dx = sel_coperct(inp_amb_dict['amb_dx']['HLTH_DX_CODE_MR_OTH'], 0.1)
varsel_amb_intrv = sel_coperct(inp_amb_dict['amb_intrv']['SE_INTRV_CODE_PR_OTH'], 0.1)

es['inp_dx']['HLTH_DX_CODE_MR_OTH'].interesting_values = varsel_inp_dx
es['inp_dx']['HLTH_DX_TYPE_CODE_MR_OTH'].interesting_values = varsel_inp_dx_type
es['inp_prvd']['Prvd_Spec_Grp_MR_Int'].interesting_values = varsel_inp_prvd
es['inp_intrv']['SE_INTRV_CODE_PR_OTH'].interesting_values = varsel_inp_intrv
es['amb_dx']['HLTH_DX_CODE_MR_OTH'].interesting_values = varsel_amb_dx
es['amb_intrv']['SE_INTRV_CODE_PR_OTH'].interesting_values = varsel_amb_intrv

/home/weijiesun/anaconda3/envs/featxgcat2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



0     b'I500'
1      b'K20'
2     b'I500'
3     b'I500'
4    b'G4020'
Name: HLTH_DX_CODE_MR_OTH, dtype: object
Number of elements : 1941839
Number of unique elements : 8882
Minimum thrshold frequency: 1941.839
Number of unique elements selected : 172

0    b'M'
1    b'M'
2    b'M'
3    b'M'
4    b'M'
Name: HLTH_DX_TYPE_CODE_MR_OTH, dtype: object
Number of elements : 1941839
Number of unique elements : 12
Minimum thrshold frequency: 1941.839
Number of unique elements selected : 7

0               b'GP'
1               b'GP'
2               b'GP'
3    b'Med_Spec_Card'
4               b'GP'
Name: Prvd_Spec_Grp_MR_Int, dtype: object
Number of elements : 451810
Number of unique elements : 9
Minimum thrshold frequency: 451.81
Number of unique elements selected : 9

0      b'3GY10VA'
1      b'3IP10VX'
2      b'2WY71HA'
3      b'3IP10VX'
4    b'2NK71BABL'
Name: SE_INTRV_CODE_PR_OTH, dtype: object
Number of elements : 224122
Number of unique elements : 5529
Minimum thrshold frequency: 224.122


In [13]:
from featuretools.primitives import TimeSincePrevious
time_since_previous_in_days = TimeSincePrevious(unit='days')

agg_prim = ['mean','n_most_common','count','percent_true','last','median','max','num_unique'] 
agg_prim = agg_prim + ['min','std','entropy','trend','all','any','first','skew']
trans_prim = [time_since_previous_in_days,'is_null','year','is_weekend']
trans_prim = trans_prim + ['percentile','diff']
where_prim = ['count','median'] 
#where_prim = where_prim + ['num_unique', 'mode'] 

features = feature_engineer(es, target_entity = 'inp_amb_obs',
    agg_prim = agg_prim,
    trans_prim = trans_prim,
    where_prim = where_prim)


/home/weijiesun/anaconda3/envs/featxgcat2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Built 12249 features
